In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
import requests
from bs4 import BeautifulSoup

In [76]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [23]:
def obtain_links_periodos(driver):
    links_asistencias = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('asistencias', link)):
            links_asistencias.append(link)
    return(links_asistencias)

def obtain_links_days(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('asisteinifinsesion', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

def obtain_partyAttendance(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('listados_asistencias', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

In [79]:
def obtain_links_periodos(main_link):
    r = requests.get(main_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    links_asistencias_sesion = []
    for i in soup.findAll("table")[0].findAll('a'):
        link = i.get('href')
        if any(re.findall('asistencias', link)):
            links_asistencias_sesion.append(main_link.split('.gob.mx')[0] + '.gob.mx'+'/' + link)
    return(links_asistencias_sesion)

def obtain_links_days(main_link):
    r = requests.get(main_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    links_asistencias_sesion = []
    partidos = []
    for i in soup.findAll("table")[0].findAll('a'):
        link = i.get('href')
        if any(re.findall('asisteinifinsesion', link)):
            links_asistencias_sesion.append(main_link.split('.gob.mx')[0] + '.gob.mx'+'/' + link)
    return(links_asistencias_sesion)

def obtain_partyAttendance(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    links_asistencias = []
    partidos = []
    for i in soup.findAll('a'):
        h = i.get('href')
        if any(re.findall('listados_asistencias', h)):
            links_asistencias.append(link.split('.gob.mx')[0] + '.gob.mx'+'/' + h)
            partidos.append(re.sub('[ ]+$', '', re.sub('^[ ]+', '', i.text)))
    return(links_asistencias, partidos)


In [55]:
def month_to_num(x):
    if x == 'Enero':
        return(1)
    elif x == 'Febrero':
        return(2)
    elif x == 'Marzo':
        return(3)
    elif x == 'Abril':
        return(4)
    elif x == 'Mayo':
        return(5)
    elif x == 'Junio':
        return(6)
    elif x == 'Julio':
        return(7)
    elif x == 'Agosto':
        return(8)
    elif x == 'Septiembre':
        return(9)
    elif x == 'Octubre':
        return(10)
    elif x == 'Noviembre':
        return(11)
    elif x == 'Diciembre':
        return(12)

In [60]:
import unidecode

In [56]:
def obtain_attendance(link, legislatura, partido):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    temp = re.sub('[ ]+$', '' , re.sub('^[ ]+', '', soup.findAll('u')[0].text))
    date_session = datetime.date(int(temp.split('-')[2]), month_to_num(temp.split('-')[1]), int(temp.split('-')[0]))
    
    
    year_legis = []
    month_legis = []
    day_legis = []
    members = []
    party = []
    type_at = []
    legis = []
    for i in soup.findAll('tr')[9:]:
        try:
            if any(re.findall('[a-z]+', i.findAll('td')[0].text)):
                continue
            else:
                members.append(unidecode.unidecode(i.findAll('td')[1].text))
                type_at.append(i.findAll('td')[2].text)
                
                party.append(partido)
                year_legis.append(date_session.year)
                month_legis.append(date_session.month)
                day_legis.append(date_session.day)
                legis.append(legislatura)
        except:
            continue
            
    return(pd.DataFrame({'legislatura':legis, 
                 'year_session':year_legis, 
                 'month_session':month_legis, 
                 'day_session':day_legis, 
                 'member':members, 
                 'party':party, 
                 'type_attendance':type_at
                 }))

    

In [80]:
link_legislaturas =[
    #'http://sitllx.diputados.gob.mx/asistencias_diputados_todosnp.php', 
   # 'http://sitllxi.diputados.gob.mx/asistencias_diputados_todosnplxi.php', 
    'http://sitllxiii.diputados.gob.mx/asistencias_diputados_todosnplxiii.php', 
   #'http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_todosnplxiv.php'
]

In [74]:
link4

'http://sitl.diputados.gob.mx/LXIV_leg/listados_asistenciasnplxiv.php?partidot=14&sesiont=104'

In [81]:
for link, leg in tqdm(zip(link_legislaturas, [63]), total = 2):
    data_legislatura = []
    
    links_periodos = obtain_links_periodos(link)
    for link2 in tqdm(links_periodos):

        links_sesiones = obtain_links_days(link2)
        for link3 in tqdm(links_sesiones):
            
            links_sesiones_partido, partido = obtain_partyAttendance(link3)
            for link4, p in zip(links_sesiones_partido, partido):
                try:
                    data_legislatura.append(obtain_attendance(link4, leg, p))
                except:
                    print(link4)
    df_attendance_leg = pd.concat(data_legislatura)
    name_file = 'floor_attendance' + str(leg) + '.csv'
    df_attendance_leg.to_csv('../../data/00-id_data/' + name_file, encoding = 'utf-8')

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  3%|▎         | 1/35 [00:02<01:23,  2.47s/it]

  6%|▌         | 2/35 [00:04<01:21,  2.46s/it]

  9%|▊         | 3/35 [00:08<01:29,  2.78s/it]

 11%|█▏        | 4/35 [00:11<01:30,  2.90s/it]

 14%|█▍        | 5/35 [00:14<01:23,  2.77s/it]

 17%|█▋        | 6/35 [00:17<01:25,  2.96s/it]

 20%|██        | 7/35 [00:19<01:18,  2.80s/it]

 23%|██▎       | 8/35 [00:22<01:12,  2.68s/it]

 26%|██▌       | 9/35 [00:24<01:07,  2.61s/it]

 29%|██▊       | 10/35 [00:27<01:04,  2.56s/it]

 31%|███▏      | 11/35 [00:29<01:01,  2.56s/it]

 34%|███▍      | 12/35 [00:32<00:58,  2.54s/it]

 37%|███▋      | 13/35 [00:34<00:55,  2.52s/it]

 40%|████      | 14/35 [00:37<00:52,  2.48s/it]

 43%|████▎     | 15/35 [00:39<00:49,  2.47s/it]

 46%|████▌     | 16/35 [00:41<00:46,  2.45s/it]

 49%|████▊     | 17/35 [00:44<00:44,  2.47s/it]

 51%|█████▏    | 18/35 [00:46<00:41,  2.46s/it]

 54%|█████▍    | 19/35 [00:49<00:39,  2.49s/it]


http://sitllxiii.diputados.gob.mx/./listados_asistenciasnplxiii.php?partidot=16&sesiont=38




 10%|█         | 3/29 [00:10<01:49,  4.22s/it]

http://sitllxiii.diputados.gob.mx/./listados_asistenciasnplxiii.php?partidot=16&sesiont=39




 14%|█▍        | 4/29 [00:13<01:31,  3.68s/it]

...



 58%|█████▊    | 19/33 [00:50<00:35,  2.52s/it]

http://sitllxiii.diputados.gob.mx/./listados_asistenciasnplxiii.php?partidot=9&sesiont=154




 61%|██████    | 20/33 [00:53<00:32,  2.47s/it]

http://sitllxiii.diputados.gob.mx/./listados_asistenciasnplxiii.php?partidot=9&sesiont=155




 64%|██████▎   | 21/33 [00:55<00:29,  2.47s/it]

 67%|██████▋   | 22/33 [00:58<00:26,  2.45s/it]

 70%|██████▉   | 23/33 [01:00<00:24,  2.47s/it]

 73%|███████▎  | 24/33 [01:03<00:22,  2.45s/it]

 76%|███████▌  | 25/33 [01:05<00:19,  2.45s/it]

 79%|███████▉  | 26/33 [01:08<00:17,  2.46s/it]

 82%|████████▏ | 27/33 [01:10<00:14,  2.44s/it]

 85%|████████▍ | 28/33 [01:12<00:12,  2.45s/it]

 88%|████████▊ | 29/33 [01:15<00:09,  2.45s/it]

 91%|█████████ | 30/33 [01:19<00:08,  2.96s/it]

 94%|█████████▍| 31/33 [01:21<00:05,  2.82s/it]

 97%|█████████▋| 32/33 [01:24<00:02,  2.72s/it]

100%|██████████| 33/33 [01:27<00:00,  2.65s/it]

 88%|████████▊ | 7/8 [07:21<01:11, 71.47s/it]

  0%|          | 0/27 [00:00<?, ?it/s]

  4%|▎         | 1/27 [00:02<01:02,  2.42s/it]

  7%|▋         | 2/27 [00:05<01:01,  2.47s/it]

 11%|█         | 3/27 [00:07<01:01,  2.56s/it]

 15%|█▍        | 4/27 [00:10<00:58,  2.54s/it]

 19%|█▊        | 5/27 [00:12<00:55,  2.52s/it]

 22%|██▏       | 6/27 [00:15<00:52

In [77]:
df_attendance_leg.type_attendance.unique()

array([' ASISTENCIA', ' CÉDULA', ' JUSTIFICADA', ' INASISTENCIA',
       ' PERMISO MESA DIRECTIVA', ' OFICIAL COMISIÓN',
       ' NO PRESENTE EN VOTACIONES'], dtype=object)